In [1]:
#from pyspark.sql import Row
#import sys
#from pyspark.sql import SparkSession

#from itertools import islice
#spark = SparkSession.builder.appName("t1").getOrCreate()
#sc = spark.sparkContext

# Part1 generating data

In [172]:
import pandas as pd
import pathlib
import datamart_geo
import datamart_profiler
import os
import json
path = pathlib.Path().absolute()

directory = os.path.join("c:\\",path) # if the file is not in c:\\, change it
dataDirectory = os.path.join(path, "Datasets") #make sure that all the csv files are in the directory named Datasets
resultDirectory = os.path.join(path, "result") #make sure that you have created the folder result

In [173]:
#print all the filenames
for root,dirs,files in os.walk(dataDirectory):
    for file in files:
        if file.endswith(".csv"):
            print(file)

2010_2016_School_Safety_Report.csv
2015_Green_Taxi_Trip_Data.csv
2015_Street_Tree_Census_Tree_Data.csv
311_Service_Requests_for_2009.csv
ACS_Community_Partners.csv
Active_Projects_Under_Construction.csv
Asbestos_Control_Program__ACP7_.csv
Board_of_Standards_and_Appeals__BSA__Applications_Status.csv
Borough_Enrollment_Offices.csv
Buildings_Selected_for_the_Alternative_Enforcement_Program__AEP_.csv
Buildings_Subject_to_HPD_Jurisdiction.csv
Bureau_of_Fire_Prevention_-_Active_Violation_Orders.csv
Bureau_of_Fire_Prevention_-_Inspections.csv
CATS_Permits.csv
Center___Service_Locations.csv
Charges.csv
City-owned_sites_that_are_available_and_potentially_suitable_for_urban_agriculture__Local_Law_46_of_2018_.csv
Construction_Demolition_Registrants.csv
Consumer_Services_Mediated_Complaints.csv
Council_Members.csv
CURRENT_BASES.csv
DCLA_Cultural_Organizations.csv
Deed_Restriction_Database.csv
Department_for_the_Aging__DFTA__All_Contracted_Providers.csv
Directory_Of_Job_Centers.csv
Directory_of_NYC

In [98]:
# read all the csv files, process them with datamart and then save it to txt files
# this step will skip files which already been processed and saved, so feel free to interrupte and continue at any time
# this cell may take several hours to run if you do not have anythin in the "result" folder 

counter = 0
df = []
for root,dirs,files in os.walk(dataDirectory):
    for file in files:
        if file.endswith(".csv"):
            done = False
            for root,dirs,files in os.walk(directory):
                if (file+".txt") in files:
                    print("skip " + file)
                    done = True
            if done == False:
                print("start loading...")
                tempd = datamart_profiler.process_dataset(dataDirectory + "/" + file)
                df.append(tempd)
                with open("result/" + file +".txt", 'w') as outfile:
                    json.dump(tempd, outfile)
                print(file)
        counter +=1
                

skip 2010_2016_School_Safety_Report.csv
skip 2015_Green_Taxi_Trip_Data.csv
skip 2015_Street_Tree_Census_Tree_Data.csv
skip 311_Service_Requests_for_2009.csv
skip ACS_Community_Partners.csv
skip Active_Projects_Under_Construction.csv
skip Asbestos_Control_Program__ACP7_.csv
skip Board_of_Standards_and_Appeals__BSA__Applications_Status.csv
skip Borough_Enrollment_Offices.csv
skip Buildings_Selected_for_the_Alternative_Enforcement_Program__AEP_.csv
skip Buildings_Subject_to_HPD_Jurisdiction.csv
skip Bureau_of_Fire_Prevention_-_Active_Violation_Orders.csv
skip Bureau_of_Fire_Prevention_-_Inspections.csv
skip CATS_Permits.csv
skip Center___Service_Locations.csv
skip Charges.csv
skip City-owned_sites_that_are_available_and_potentially_suitable_for_urban_agriculture__Local_Law_46_of_2018_.csv
skip Construction_Demolition_Registrants.csv
skip Consumer_Services_Mediated_Complaints.csv
skip Council_Members.csv
skip CURRENT_BASES.csv
skip DCLA_Cultural_Organizations.csv
skip Deed_Restriction_Data

In [174]:
#count the number of dataset which contains lat lon based on semantic types
counter = 0
djson = []
filenames = []
for root,dirs,files in os.walk(resultDirectory):
    for file in files:
        if file.endswith(".txt"):
            with open("result/" + file) as f:
                tempdjson = json.load(f)
                djson.append(tempdjson) 
                filenames.append(file) #match filename with djson content
def check_lat(col_list):
    for content in col_list:
        if 'semantic_types' in content:
            if content['semantic_types'] ==['http://schema.org/latitude']:
                return True
    return False
def check_lon(col_list):
    for content in col_list:
        if 'semantic_types' in content:
            if content['semantic_types'] ==['http://schema.org/longitude']:
                return True
    return False
num_data = len(djson)
counter = 0
latlon_data = pd.DataFrame({'latlon' : []})
for i in range(num_data):
    if check_lat(djson[i]["columns"]) and check_lon(djson[i]["columns"]):
        counter +=1
        latlon_data.loc[filenames[i]] = [1]
    elif check_lat(djson[i]["columns"]) or check_lon(djson[i]["columns"]):
        print(djson[i])
    else:
        latlon_data.loc[filenames[i]] = [0]
        
counter       

94

In [175]:
#save the information of column names into a csv file
#1 means this csv has this column
#0 means this csv do not have this column

names = []
for i in range(num_data):
    for content in djson[i]["columns"]:
        name = content["name"].lower()
        if name not in names:
            names.append(name)      
names = sorted(names)
zeros = [0]*len(names)
df_names = pd.DataFrame(columns = names)
for i in range(num_data):
    df_names.loc[filenames[i]] = zeros
    for content in djson[i]["columns"]:
        name = content["name"].lower()
        df_names.loc[filenames[i]][name] = 1

In [176]:
df_names.to_csv("column_names.csv")

In [177]:
df_names

,comparable rental 1 boro-block-lot,consumption (gj),consumption (therms),# days,# isps,# of b/c violations at start,# of carriers,# schools,1-br units,2-br units,...,zone,zone dist 1,zone dist 2,zoning dist1,zoning dist2,zoning dist3,zoning distr 1,zoning distr 2,zoning distr 3,zoning district
2010_2016_School_Safety_Report.csv.txt,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2015_Green_Taxi_Trip_Data.csv.txt,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2015_Street_Tree_Census_Tree_Data.csv.txt,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
311_Service_Requests_for_2009.csv.txt,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ACS_Community_Partners.csv.txt,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Voting_Poll_Sites.csv.txt,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Water_Consumption_And_Cost__2013_-_2020_.csv.txt,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Wholesale_Markets.csv.txt,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
WiredNYC___All_Buildings_Data.csv.txt,0,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [178]:
#As you can see there are some different column names which contain the same information such as "zip" and "zip code"
df_names.sum().sort_values(ascending=False)[:50]

latitude              94.0
longitude             94.0
borough               91.0
bin                   78.0
community board       70.0
nta                   68.0
bbl                   68.0
census tract          64.0
postcode              58.0
council district      53.0
city                  35.0
address               28.0
state                 27.0
lot                   26.0
block                 25.0
street                21.0
location              19.0
status                16.0
boro                  14.0
phone                 14.0
zip                   14.0
street name           13.0
number                11.0
location 1            11.0
name                  10.0
zip code              10.0
council district       8.0
email                  8.0
street address         8.0
expiration date        7.0
account name           7.0
jurisdiction_code      7.0
agency                 7.0
description            6.0
comments               6.0
building               6.0
community council      6.0
a

# Part2 search word from columns

In [179]:
def search_words(words):
    names = []
    for name in df_names.columns:
        if words.lower() in name.lower():
            names.append(name)
    filenames = {}
    counter = 0
    for name in names:
        indexlist = list(df_names[df_names[name] == 1].index)
        filenames[name]  = indexlist
        counter += len(indexlist)
    print("Number of columns containing this words: " + str(counter))     
    return filenames

# 

In [180]:
#simple count
temp = search_words("zip")

Number of columns containing this words: 53


In [181]:
#more detailed result
temp

{'address zip': ['Legally_Operating_Businesses.csv.txt'],
 'business zip': ['Consumer_Services_Mediated_Complaints.csv.txt'],
 'city, state, zip ': ['Revised_Notice_of_Property_Value__RNOPV_.csv.txt'],
 'complainant zip ': ['Consumer_Services_Mediated_Complaints.csv.txt'],
 'eviction_zip': ['Evictions.csv.txt'],
 'filing representative zip': ['DOB_NOW__Build___Job_Application_Filings.csv.txt'],
 'garage_zip': ['Routes.csv.txt'],
 'incident zip': ['311_Service_Requests_for_2009.csv.txt'],
 'incident_address_zip': ['DOHMH_Indoor_Environmental_Complaints.csv.txt'],
 'mailingzip': ['M_WBE__LBE__and_EBE_Certified_Business_List.csv.txt'],
 'owner zip code': ['DOB_NOW_Build_Approved_Permits.csv.txt'],
 'owner_zip': ['DOB_NOW__Safety___Facades_Compliance_Filings.csv.txt'],
 'practice zip code': ['NYC_REACH_Members.csv.txt'],
 'program zipcode': ['Department_for_the_Aging__DFTA__All_Contracted_Providers.csv.txt'],
 'property disposed to zip': ['Deed_Restriction_Database.csv.txt'],
 'qewi_zip': 

In [182]:
#get the json file from certain filename
def json_from_filename(filename):
    return djson[filenames.index(filename)]

In [183]:
def get_result(word):
    temp = search_words(word)
    info = []

    for key in temp:
        values = temp[key]
        for filename in values:
            jsoncontent = json_from_filename(filename)
            for i in jsoncontent["columns"]:
                if i["name"].lower()  == key:
                    info.append((i,filename))
    keys = []
    for dict_i,filename in info:
        for key in dict_i.keys():
            if key not in keys:
                keys.append(key)   
    result = pd.DataFrame(columns = keys)
    [None]*len(result.columns)
    for dict_i,filename in info:
        result.loc[filename] = [None]*len(result.columns)
        for key in dict_i.keys():
            if key in result.columns:
                result.loc[filename][key] = dict_i[key]
    print("Number of files which have columns containing this words: " + str(len(result)))
    print("Number of all files:" + str(len(filenames)))
    result.to_csv(word+".csv")
    return result

In [184]:
result = get_result("zip")

Number of columns containing this words: 53
Number of files which have columns containing this words: 48
Number of all files:138


In [185]:
result = get_result("latitude")

Number of columns containing this words: 98
Number of files which have columns containing this words: 96
Number of all files:138


In [187]:
with open("data/metadata.json") as f:
     actual = json.load(f)

In [188]:
#count the number of dataset which contains lat lon based on semantic types
counter = 0
djson = []
filenames = []
for root,dirs,files in os.walk(resultDirectory):
    for file in files:
        if file.endswith(".txt"):
            with open("result/" + file) as f:
                tempdjson = json.load(f)
                djson.append(tempdjson) 
                filenames.append(file) #match filename with djson content
def check_lat(col_list):
    for content in col_list:
        if 'semantic_types' in content:
            if content['semantic_types'] ==['http://schema.org/latitude']:
                return True
    return False
def check_lon(col_list):
    for content in col_list:
        if 'semantic_types' in content:
            if content['semantic_types'] ==['http://schema.org/longitude']:
                return True
    return False
num_data = len(djson)
counter = 0
latlon_data = pd.DataFrame({'latlon' : []})
for i in range(num_data):
    if check_lat(djson[i]["columns"]) and check_lon(djson[i]["columns"]):
        counter +=1
        latlon_data.loc[filenames[i]] = [1]
    elif check_lat(djson[i]["columns"]) or check_lon(djson[i]["columns"]):
        print(djson[i])
    else:
        latlon_data.loc[filenames[i]] = [0]
        
counter       

94

In [279]:
def get_positive(words_list):
    positive = []
    for words in words_list:
        temp = search_words(words)
        for i in temp:
            for j in temp[i]:
                #print(j)
                positive.append(j.split(".txt")[0])
    return positive


In [211]:
len(get_positive(["latitude"]))

Number of columns containing this words: 98


98

In [231]:
positive = get_positive(["zip"])

Number of columns containing this words: 53


In [233]:
search_words("zip")

Number of columns containing this words: 53


{'address zip': ['Legally_Operating_Businesses.csv.txt'],
 'business zip': ['Consumer_Services_Mediated_Complaints.csv.txt'],
 'city, state, zip ': ['Revised_Notice_of_Property_Value__RNOPV_.csv.txt'],
 'complainant zip ': ['Consumer_Services_Mediated_Complaints.csv.txt'],
 'eviction_zip': ['Evictions.csv.txt'],
 'filing representative zip': ['DOB_NOW__Build___Job_Application_Filings.csv.txt'],
 'garage_zip': ['Routes.csv.txt'],
 'incident zip': ['311_Service_Requests_for_2009.csv.txt'],
 'incident_address_zip': ['DOHMH_Indoor_Environmental_Complaints.csv.txt'],
 'mailingzip': ['M_WBE__LBE__and_EBE_Certified_Business_List.csv.txt'],
 'owner zip code': ['DOB_NOW_Build_Approved_Permits.csv.txt'],
 'owner_zip': ['DOB_NOW__Safety___Facades_Compliance_Filings.csv.txt'],
 'practice zip code': ['NYC_REACH_Members.csv.txt'],
 'program zipcode': ['Department_for_the_Aging__DFTA__All_Contracted_Providers.csv.txt'],
 'property disposed to zip': ['Deed_Restriction_Database.csv.txt'],
 'qewi_zip': 

In [232]:
positive

['Legally_Operating_Businesses.csv',
 'Consumer_Services_Mediated_Complaints.csv',
 'Revised_Notice_of_Property_Value__RNOPV_.csv',
 'Consumer_Services_Mediated_Complaints.csv',
 'Evictions.csv',
 'DOB_NOW__Build___Job_Application_Filings.csv',
 'Routes.csv',
 '311_Service_Requests_for_2009.csv',
 'DOHMH_Indoor_Environmental_Complaints.csv',
 'M_WBE__LBE__and_EBE_Certified_Business_List.csv',
 'DOB_NOW_Build_Approved_Permits.csv',
 'DOB_NOW__Safety___Facades_Compliance_Filings.csv',
 'NYC_REACH_Members.csv',
 'Department_for_the_Aging__DFTA__All_Contracted_Providers.csv',
 'Deed_Restriction_Database.csv',
 'DOB_NOW__Safety___Facades_Compliance_Filings.csv',
 'DOB_ECB_Violations.csv',
 '311_Service_Requests_for_2009.csv',
 'Buildings_Subject_to_HPD_Jurisdiction.csv',
 'Charges.csv',
 'DOB_Job_Application_Filings.csv',
 'DOB_NOW__Build___Job_Application_Filings.csv',
 'Housing_Maintenance_Code_Complaints.csv',
 'IDNYC_Locations.csv',
 'Inspections.csv',
 'License_Applications.csv',
 'M_W

In [237]:
def get_acc(subwords, words):
    positive = get_positive(subwords)

    TP = 0
    TN = 0
    FP = 0
    FN = 0
    for file in filenames:  
        file2 = file.split(".txt")[0].replace("-","_").replace("___","_").replace("__","_").replace("_.csv",".csv").replace("NYPD_Shooting_Incident_Data_Historic.csv","NYPD_Shooting_Incident_Data_Historic_.csv")
        a = 0
        b = 0
        for i in actual[file2]["attributes"]:
            if(actual[file2]["attributes"][i]["type"]) == words:
                a = 1
            if file.split(".txt")[0] in positive:
                b = 1
        if a == 1 and b== 1:
            TP +=1
        elif a == 0 and b == 0:
            TN +=1
        elif a == 1 and b == 0:
            FN+=1
        elif a == 0 and b ==1:
            FP +=1
    print("TP"+str(TP))
    print("TN"+str(TN))
    print("FP"+str(FP))
    print("FN"+str(FN))
    print("Acc: "+str((TP+TN)/(TP+TN+FN+FP)))
    print("Recall: "+str(TP/(TP+FN)))

        

In [238]:
get_acc(["latitude"],"latitude")
#some can be identified from column name directly

Number of columns containing this words: 98
TP96
TN42
FP0
FN0
Acc: 1.0
Recall: 1.0


In [239]:
get_acc(["zip","post"],"zipcode")
#some has errors. For example, a column named "Property Disposed to ZIP" is not ZIP
#need to identify other features

Number of columns containing this words: 53
Number of columns containing this words: 62
TP103
TN31
FP4
FN0
Acc: 0.9710144927536232
Recall: 1.0


positive = get_positive(["zip","post"])
temp = search_words("zip")
temp
for filename in positive:
    for key in temp:
        for names in temp[key]:
            if names == filename+".txt":
                jfile = json_from_filename(filename+".txt")
                for i in jfile["columns"]:
                    if i["name"] == key:
                        print(i["structural_type"] == 'http://schema.org/Integer')

In [242]:
jfile

{'size': 62289243,
 'nb_rows': 266686,
 'average_row_size': 233.56772758974975,
 'nb_profiled_rows': 214089,
 'nb_columns': 27,
 'columns': [{'name': 'DCA License Number',
   'structural_type': 'http://schema.org/Text',
   'semantic_types': [],
   'num_distinct_values': 214087},
  {'name': 'License Type',
   'structural_type': 'http://schema.org/Text',
   'semantic_types': ['http://schema.org/Enumeration'],
   'num_distinct_values': 2},
  {'name': 'License Expiration Date',
   'structural_type': 'http://schema.org/Text',
   'semantic_types': ['http://schema.org/Enumeration',
    'http://schema.org/DateTime'],
   'missing_values_ratio': 0.06251605640644779,
   'num_distinct_values': 2236,
   'mean': 1413769524.64961,
   'stddev': 175809463.35037568,
   'coverage': [{'range': {'gte': 1077926400.0, 'lte': 1262217600.0}},
    {'range': {'gte': 1293753600.0, 'lte': 1459382400.0}},
    {'range': {'gte': 1488240000.0, 'lte': 1648684800.0}}],
   'temporal_resolution': 'day'},
  {'name': 'Licen

In [220]:
get_acc(["city"],"city")
#FP because of name of electricity

Number of columns containing this words: 68
TP42
TN83
FP12
FN1
Acc: 0.9057971014492754
Recall: 0.9767441860465116


In [222]:
get_acc(["state"],"state")

Number of columns containing this words: 43
TP34
TN100
FP4
FN0
Acc: 0.9710144927536232
Recall: 1.0


In [226]:
get_acc(["boro"],"borough")

Number of columns containing this words: 152
TP118
TN18
FP2
FN0
Acc: 0.9855072463768116
Recall: 1.0


# ignore the following cells

In [ ]:
>>> geo_data = datamart_geo.GeoData.download()  # Download data if needed

>>> france = geo_data.resolve_name('France')
>>> france
<datamart_geo.Area "Republic of France" (3017382) type=Type.ADMIN_0>
>>> france.latitude, france.longitude
(46.0, 2.0)
>>> france.bounds
(-61.797841, 55.854503, -21.370782, 51.087541)
>>> assert france.type == datamart_geo.Type.ADMIN_0
>>> assert france.type == datamart_geo.Type.COUNTRY

>>> cuers = geo_data.resolve_name('Cuers')
>>> cuers
<datamart_geo.Area "Cuers" (6451482) type=Type.ADMIN_4>
>>> cuers.latitude, cuers.longitude
(43.2375, 6.07083)
>>> cuers.get_parent_area()
<datamart_geo.Area "Arrondissement de Toulon" (2972326) type=Type.ADMIN_3>

>>> [
...     # Show multiple results with their parent ADMIN_1
...     (a, a.get_parent_area(datamart_geo.Type.ADMIN_1))
...     for a in geo_data.resolve_name_all('Var')
... ]
[(<datamart_geo.Area "Var" (2970749) type=Type.ADMIN_2>,
  <datamart_geo.Area "Provence-Alpes-Côte d'Azur" (2985244) type=Type.ADMIN_1>),
 (<datamart_geo.Area "Vars" (6427887) type=Type.ADMIN_4>,
  <datamart_geo.Area "Nouvelle-Aquitaine" (11071620) type=Type.ADMIN_1>),
 (<datamart_geo.Area "Vars" (6442138) type=Type.ADMIN_4>,
  <datamart_geo.Area "Bourgogne-Franche-Comté" (11071619) type=Type.ADMIN_1>)]
